In [11]:
from sleeper_wrapper import League

league = League('918363762489450496')


In [13]:
completion = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": "Hello world"}])
print(completion.choices[0].message.content)

Hello! How can I assist you today?


In [9]:
d_users = league.get_users()
users = {}
for r in d_users:
    users[r['user_id']] = {'user_id': r['user_id'], 'name': r['display_name']}

In [10]:
users

{'95259455910199296': {'user_id': '95259455910199296', 'name': 'Deign'},
 '413881572480000000': {'user_id': '413881572480000000', 'name': 'anushm'},
 '467082184046407680': {'user_id': '467082184046407680', 'name': 'tejas'},
 '467082433204842496': {'user_id': '467082433204842496', 'name': 'snethi'},
 '467082675589476352': {'user_id': '467082675589476352', 'name': 'rbada'},
 '467416275337867264': {'user_id': '467416275337867264', 'name': 'vish3418'},
 '467974784802615296': {'user_id': '467974784802615296',
  'name': 'psychomaragiri'},
 '470020787244363776': {'user_id': '470020787244363776', 'name': 'preraku'},
 '471181259578535936': {'user_id': '471181259578535936', 'name': 'shwanrat'},
 '730218826838429696': {'user_id': '730218826838429696', 'name': 'JCH4'},
 '730543392060231680': {'user_id': '730543392060231680', 'name': 'suprith'},
 '733202403196780544': {'user_id': '733202403196780544', 'name': 'Shang_Chi'}}

In [11]:
p = league.get_rosters()
week = 0
roster = []
new_user = {}
for i, x in enumerate(p):
    oID = x['owner_id']
    rec = users[oID]
    rec['roster_id'] = i + 1
    rec['wins'] = x['settings']['wins']
    rec['losses'] = x['settings']['losses']
    points = int(x['settings']['fpts']) + (int(x['settings']['fpts_decimal']) / 100)
    pa = int(x['settings']['fpts_against']) + (int(x['settings']['fpts_against_decimal']) / 100)
    rec['pf'] = points
    rec['pa'] = pa
    roster.append(rec['name'])
    week = rec['wins'] + rec['losses']
    if not week == 0:
        rec['avg_pf'] = points / float(week)
        rec['avg_pa'] = pa / float(week)
    else:
        rec['avg_pf'] = 0
        rec['avg_pa'] = 0
        
    new_user[rec['roster_id']] = rec

In [12]:
roster

['anushm',
 'psychomaragiri',
 'tejas',
 'rbada',
 'Deign',
 'snethi',
 'shwanrat',
 'preraku',
 'suprith',
 'JCH4',
 'Shang_Chi',
 'vish3418']

In [13]:
new_user

{1: {'user_id': '413881572480000000',
  'name': 'anushm',
  'roster_id': 1,
  'wins': 0,
  'losses': 1,
  'pf': 64.32,
  'pa': 84.44,
  'avg_pf': 64.32,
  'avg_pa': 84.44},
 2: {'user_id': '467974784802615296',
  'name': 'psychomaragiri',
  'roster_id': 2,
  'wins': 0,
  'losses': 1,
  'pf': 69.9,
  'pa': 87.14,
  'avg_pf': 69.9,
  'avg_pa': 87.14},
 3: {'user_id': '467082184046407680',
  'name': 'tejas',
  'roster_id': 3,
  'wins': 0,
  'losses': 1,
  'pf': 53.18,
  'pa': 90.62,
  'avg_pf': 53.18,
  'avg_pa': 90.62},
 4: {'user_id': '467082675589476352',
  'name': 'rbada',
  'roster_id': 4,
  'wins': 1,
  'losses': 0,
  'pf': 84.44,
  'pa': 64.32,
  'avg_pf': 84.44,
  'avg_pa': 64.32},
 5: {'user_id': '95259455910199296',
  'name': 'Deign',
  'roster_id': 5,
  'wins': 0,
  'losses': 1,
  'pf': 68.7,
  'pa': 96.46,
  'avg_pf': 68.7,
  'avg_pa': 96.46},
 6: {'user_id': '467082433204842496',
  'name': 'snethi',
  'roster_id': 6,
  'wins': 1,
  'losses': 0,
  'pf': 74.54,
  'pa': 59.62,
 

In [71]:
from collections import defaultdict
import pandas as pd

roster_conversions = ['Mattapalli', 'Komee', 'Idate', 'Bada', 'Digby', 'Nethi', 'Rattan', 'Upadhyaya', 'Aireddy', 'Hansen', 'Le', 'Pandya']

def get_matchups(week: int) -> (defaultdict, pd.DataFrame):
    all_matchups = league.get_matchups(week)
    matchups = defaultdict(list)
    ranks = []
    for matchup in all_matchups:
        matchup_id = matchup['matchup_id']
        matchups[matchup_id].append((matchup['roster_id'], matchup['points']))
        ranks.append([matchup['roster_id'], matchup['points']])
        
    ranks_df = pd.DataFrame(ranks, columns=['Id', 'Score']).sort_values(by='Score', ascending=False).set_index('Id')
    ranks_df['Rank'] = range(1, 13)
    return matchups, ranks_df
    
def get_scores(week: int) -> list:
    matchups, ranks = get_matchups(week)
    df = []
    for j in matchups.values():
        roster_id1 = j[0][0]
        roster_id2 = j[1][0]
        one_score = j[0][1]
        two_score = j[1][1]
        one_diff = round(one_score - two_score, 2)
        two_diff = round(two_score - one_score, 2)
        df.append([2023, roster_conversions[roster_id1-1], week, 1 if one_score > two_score else 0, one_score, two_score, roster_conversions[roster_id2-1], one_diff, ranks.loc[roster_id1, 'Rank']])
        df.append([2023, roster_conversions[roster_id2-1], week, 1 if one_score < two_score else 0, two_score, one_score, roster_conversions[roster_id1-1], two_diff, ranks.loc[roster_id2, 'Rank']])
        
    return df


In [72]:
get_scores(1)

[[2023, 'Mattapalli', 1, 0, 64.32, 84.44, 'Bada', -20.12, 9],
 [2023, 'Bada', 1, 1, 84.44, 64.32, 'Mattapalli', 20.12, 5],
 [2023, 'Komee', 1, 0, 69.9, 87.14, 'Upadhyaya', -17.24, 7],
 [2023, 'Upadhyaya', 1, 1, 87.14, 69.9, 'Komee', 17.24, 4],
 [2023, 'Idate', 1, 0, 53.18, 90.62, 'Hansen', -37.44, 12],
 [2023, 'Hansen', 1, 1, 90.62, 53.18, 'Idate', 37.44, 3],
 [2023, 'Digby', 1, 0, 68.7, 96.46, 'Rattan', -27.76, 8],
 [2023, 'Rattan', 1, 1, 96.46, 68.7, 'Digby', 27.76, 2],
 [2023, 'Nethi', 1, 1, 74.54, 59.62, 'Aireddy', 14.92, 6],
 [2023, 'Aireddy', 1, 0, 59.62, 74.54, 'Nethi', -14.92, 11],
 [2023, 'Le', 1, 1, 131.64, 61.7, 'Pandya', 69.94, 1],
 [2023, 'Pandya', 1, 0, 61.7, 131.64, 'Le', -69.94, 10]]